In [1]:
#!/usr/bin/env python
import numpy as np
import pandas as pd
from wellborecalc import calc_casing as cc, calc_interval as ci, calc_pump as cp
from utils.utils import getlogger, validate, find_nearest_value, find_next_largest_value

logger = getlogger()


In [2]:
class WellBoreDict:
    #TODO: add casing_stages attribute
    data_param_names = [
        #----------------input
        "casing_diameter_data",
        "drilling_diameter_data",
        "casing_recommendid_bit_data",
        "depth_data",
        #----------------tbd by pipeline
        "casing_stage_data"
    ]

    initial_param_names = [
        "required_flow_rate",
        "hydraulic_conductivity",
        "average_porosity",
        "bore_lifetime_year",
        "groundwater_depth",
        "long_term_decline_rate",
        "allowable_drawdown",
        "safety_margin",
        #Below are derived from the above parameters
        "depth_to_top_screen",
        "required_flow_rate_per_litre_sec",
        "required_flow_rate_per_m3_sec",
        "bore_lifetime_per_day",
        "aquifer_thickness",
        "depth_to_aquifer_base",
        #And below are pamams with preassigned default values
        "sand_face_velocity_production",
        "sand_face_velocity_injection",
        "net_to_gross_ratio_aquifer",
        "aquifer_average_porosity",
        "pipe_roughness_coeff"
    ]

    interval_param_names = [
        "production_screen_length",
        "injection_screen_length",
        "production_screen_length_error",
        "injection_screen_length_error",
        "production_open_hole_diameter",
        "injection_open_hole_diameter",
    ]

    pump_param_names = [
        "pump_inlet_depth",
        "minimum_pump_housing_diameter",
    ]

    

    def __init__(self, logger=None):
        self.logger = logger
        self.casing_diameter_data = pd.DataFrame(columns=['inches', 'metres'])
        self.drilling_diameter_data = pd.DataFrame(
            columns=['inches', 'metres'])
        self.casing_recommendid_bit_data = pd.DataFrame(
            columns=['metres', 'recommended_bit'])
        self.depth_data = pd.DataFrame(
            columns=["aquifer_layer", "is_aquifer", "depth_to_base"])
        self.required_flow_rate = None
        self.hydraulic_conductivity = None
        self.average_porosity = None
        self.bore_lifetime_year = None
        self.groundwater_depth = None
        self.long_term_decline_rate = None
        self.allowable_drawdown = None
        self.safety_margin = None
        # self.is_production = None #additional handling - don't need it
        #----------------------------------------------------------------
        self.required_flow_rate_per_litre_sec = None
        self.required_flow_rate_per_m3_sec = None
        self.bore_lifetime_per_day = None
        self.depth_to_top_screen = None
        self.aquifer_thickness = None
        self.depth_to_aquifer_base = None
        #----------------------------------------------------------------
        self.sand_face_velocity_production= .01
        self.sand_face_velocity_injection = .003
        self.net_to_gross_ratio_aquifer = 1
        self.aquifer_average_porosity = .25
        self.pipe_roughness_coeff = 100
        #----------------------------------------------------------------
        self.initialise_casing_stage_data()

    def set_diameter_data(self,
                          casing_diameter_data=None,
                          drilling_diameter_data=None,
                          casing_recommended_bit_data=None):
        self.casing_diameter_data = casing_diameter_data or pd.DataFrame({
            'inches': [ 4, 4.5, 5, 5.5, 6.625, 7, 8.625, 9.625, 10.75, 13.375, 18.625, 20, 24, 30],
            'metres': [ 0.1016, 0.1143, 0.127, 0.1397, 0.168275, 0.1778, 0.219075, 0.244475, 0.27305, 0.339725, 0.473075, 0.508, 0.6096, 0.762]
        })

        self.drilling_diameter_data = drilling_diameter_data or pd.DataFrame({
            'inches': [ 7.5, 8.5, 9, 9.5, 10.625, 11.625, 12.25, 13.75, 15, 16, 17.5, 18.5, 20, 22, 24, 26, 28, 30, 32, 34, 36],
            'metres': [ 0.1905, 0.2159, 0.2286, 0.2413, 0.269875, 0.295275, 0.31115, 0.34925, 0.381, 0.4064, 0.4445, 0.4699, 0.508, 0.5588, 0.6096, 0.6604, 0.7112, 0.762, 0.8128, 0.8636, 0.9144]
        })

        self.casing_recommendid_bit_data = casing_recommended_bit_data or pd.DataFrame({
            'metres': [0.000000, 0.101600, 0.114300, 0.127000, 0.139700, 0.168275, 0.177800, 0.219075, 0.244475, 0.273050, 0.339725, 0.473075, 0.508000, 0.609600, 0.762000],
            'recommended_bit': [0.190500, 0.190500, 0.215900, 0.215900, 0.228600, 0.269875, 0.269875, 0.311150, 0.349250, 0.381000, 0.444500, 0.609600, 0.609600, 0.711200, 0.914400]
        })

    def set_depth_data(self, depth_data):
        if not isinstance(depth_data, pd.DataFrame):
            try:
                depth_data_pd = pd.DataFrame(depth_data)
            except ValueError as e:
                logger.error(e)
        else:
            depth_data_pd = depth_data.copy()
        columns = ["aquifer_layer", "is_aquifer", "depth_to_base"]
        depth_data_pd.columns = columns
        self.depth_data = depth_data_pd.set_index(
            "aquifer_layer")

    def set_params(self, arg_names, **kwargs):
        for arg_name in arg_names:
            value = kwargs.get(arg_name)
            if value is not None:
                setattr(self, arg_name, value)

    # def set_is_production(self, is_production):
    #     self.is_production = is_production

    def get_casing_diameters(self, as_numpy=True):
        """Returns as numpy array """
        diam = self.casing_diameter_data['metres']
        return np.array(diam) if as_numpy else diam

    def get_drilling_diameters(self, as_numpy=True):
        diam = self.drilling_diameter_data['metres']
        return np.array(diam) if as_numpy else diam

    def initialise_casing_stage_data(self):
        casing_stages = ["pre_collar",
                         "superficial_casing",
                         "pump_chamber_casing",
                         "intermediate_casing",
                         "screen_riser",
                         "production_screen"]
        casing_columns=['top','bottom','casing','drill_bit']
        nan_array = np.full((len(casing_stages), len(casing_columns)), np.nan)
        casing_df = pd.DataFrame(nan_array, 
                                 columns=casing_columns).set_index(pd.Index(casing_stages, name='casing_stages'))
        self.casing_stage_data = casing_df

    def initialise(self, **kwargs):
        self.set_diameter_data()
        self.set_depth_data(kwargs.get('depth_data'))
        self.set_params(self.initial_param_names, **kwargs)
        # self.set_is_production(kwargs.get("is_production"))
        self.required_flow_rate_per_litre_sec = self.required_flow_rate / 86.4
        self.required_flow_rate_per_m3_sec = self.required_flow_rate / 86400
        self.bore_lifetime_per_day = self.bore_lifetime_year * 365
        #print(self.depth_data)
        self.depth_to_top_screen = self.depth_data.loc['LMTA']['depth_to_base']
        self.aquifer_thickness = self.depth_data.loc['LTA']['depth_to_base'] - self.depth_to_top_screen
        self.depth_to_aquifer_base = self.depth_data.loc['QA_UTQA']['depth_to_base']

    def validate_data(self):
        for arg in self.data_param_names:
            value = getattr(self, arg)
            if validate(value, lambda x: not isinstance(x, pd.DataFrame) or x.empty):
                logger.critical('validate_data failed')
                return False
        return True
    
    

    def validate_initial_inputs(self):
        """ 
        #TODO: write in DRY util method
        Performs validation checks for initial input data
        """
        if not isinstance(self.casing_diameter_data, pd.DataFrame) or self.casing_diameter_data.empty:
            self.logger.error("Invalid or missing casing diameter data")
            return False

        if not isinstance(self.drilling_diameter_data, pd.DataFrame) or self.drilling_diameter_data.empty:
            self.logger.error("Invalid or missing drilling diameter data")
            return False

        if not isinstance(self.casing_recommendid_bit_data, pd.DataFrame) or self.casing_recommendid_bit_data.empty:
            self.logger.error("Invalid or missing casing recommended bit data")
            return False

        if not isinstance(self.depth_data, pd.DataFrame) or self.depth_data.empty:
            self.logger.error("Invalid or missing depth data")
            return False

        if not all([
            self.required_flow_rate,
            self.hydraulic_conductivity,
            self.average_porosity,
            self.bore_lifetime_year,
            self.groundwater_depth,
            self.long_term_decline_rate,
            self.allowable_drawdown,
            self.safety_margin
        ]):
            self.logger.error("Invalid or missing float input data")
            return False
        

        return True

In [3]:
class CalcPipeline:
    # TODO: Add validation checks for the inputs
    def __init__(self, wellboredict: WellBoreDict):
        self.wbd = wellboredict  # must be a fully initialised instance

    def interval_pipeline(self):
        wbd = self.wbd
        casing_diameters = wbd.get_casing_diameters()
        drilling_diameters = wbd.get_drilling_diameters()
        interval_df = pd.DataFrame(casing_diameters,
                                   columns=['prod_casing_diameters'])
        # storing interval stage results
        ir = {param_name: None for param_name in wbd.interval_param_names}

        # print(prod_casing_diameters)
        ir['production_screen_length'], ir['production_screen_length_error'] = ci.calculate_minimum_screen_length(wbd.required_flow_rate,
                                                                                                                  wbd.hydraulic_conductivity,
                                                                                                                  wbd.bore_lifetime_per_day,
                                                                                                                  wbd.aquifer_thickness,
                                                                                                                  False)
        ir['injection_screen_length'], ir['injection_screen_length_error'] = ci.calculate_minimum_screen_length(wbd.required_flow_rate,
                                                                                                                wbd.hydraulic_conductivity,
                                                                                                                wbd.bore_lifetime_per_day,
                                                                                                                wbd.aquifer_thickness,
                                                                                                                True)
        interval_df['casing_frictions'] = ci.calculate_casing_friction(wbd.depth_to_top_screen,
                                                                       wbd.required_flow_rate_per_m3_sec,
                                                                       casing_diameters,
                                                                       wbd.pipe_roughness_coeff)
        interval_df['production_minimum_screen_diameters'] = ci.calculate_minimum_screen_diameter(interval_df['casing_frictions'].to_numpy(),
                                                                                                  screen_length=ir[
                                                                                                      'production_screen_length'],
                                                                                                  req_flow_rate=wbd.required_flow_rate_per_m3_sec,
                                                                                                  pipe_roughness_coeff=wbd.pipe_roughness_coeff
                                                                                                  )

        interval_df['production_screen_diameters'] = interval_df.apply(lambda row: find_next_largest_value(row['production_minimum_screen_diameters'],
                                                                                                           casing_diameters) if not np.isnan(row['production_minimum_screen_diameters']) else np.nan,
                                                                       axis=1)
        interval_df['total_casing'] = interval_df.apply(lambda row: ci.calculate_total_casing(row['prod_casing_diameters'],
                                                                                              row['production_screen_diameters'],
                                                                                              wbd.depth_to_top_screen-10,  # TODO:fix later)
                                                                                              ir['production_screen_length']),
                                                        axis=1)
        ir['best_option'] = interval_df.iloc[interval_df['total_casing'].argmin(
            skipna=True)]['prod_casing_diameters']
        ohd_min_production = ci.calculate_minimum_open_hole_diameter(wbd.required_flow_rate_per_m3_sec,
                                                                     ir['production_screen_length'],
                                                                     wbd.sand_face_velocity_production,
                                                                     wbd.aquifer_average_porosity,
                                                                     wbd.net_to_gross_ratio_aquifer)
        ohd_min_injection = ci.calculate_minimum_open_hole_diameter(wbd.required_flow_rate_per_m3_sec,
                                                                    ir['injection_screen_length'],
                                                                    wbd.sand_face_velocity_injection,
                                                                    wbd.aquifer_average_porosity,
                                                                    wbd.net_to_gross_ratio_aquifer)

        # print(ohd_min_production, ohd_min_injection)
        ir['production_open_hole_diameter'] = find_next_largest_value(
            ohd_min_production, drilling_diameters)
        ir['injection_open_hole_diameter'] = find_next_largest_value(
            ohd_min_injection, drilling_diameters)
        # print(interval_df['production_screen_diameters'])
        wbd.set_params(ir.keys(), **ir)
        for key, value in ir.items():
            print(f"{key}: {value}")
        # print(interval_df)
        wbd.interval_data = interval_df

    def pump_pipeline(self):
        # pump_results
        wbd = self.wbd
        pr = {param_name: None for param_name in wbd.pump_param_names}
        pr['pump_inlet_depth'] = cp.calculate_pump_inlet_depth(wbd.groundwater_depth,
                                                               wbd.allowable_drawdown,
                                                               wbd.safety_margin,
                                                               wbd.long_term_decline_rate,
                                                               wbd.bore_lifetime_year)
        pump_diameter = cp.assign_pump_diameter(
            wbd.required_flow_rate_per_litre_sec)
        pr['minimum_pump_housing_diameter'] = cp.calculate_minimum_pump_housing_diameter(wbd.required_flow_rate_per_m3_sec,
                                                                                         pump_diameter
                                                                                         )
        wbd.set_params(pr.keys(), **pr)
        for key, value in pr.items():
            print(f"{key}: {value}")

    def casing_pipeline(self):
        wbd = self.wbd
        print(wbd.casing_stage_data)
        casing_diameters = wbd.get_casing_diameters()
        # drilling_diameters = wbd.get_drilling_diameters()
        casing_stage_data = wbd.casing_stage_data.copy()
        superficial_casing_required = cc.is_superficial_casing_required(wbd.depth_to_aquifer_base)
        #------superficial_casing
        casing_stage_data.loc['superficial_casing'] = [*cc.calculate_superficial_casing_depths(superficial_casing_required,
                                                                                wbd.depth_to_aquifer_base),
                                           cc.calculate_superficial_casing_diameter(superficial_casing_required)]
        #------pre_collar
        casing_stage_data['pre_collar', ['top', 'bottom','casing']] = [
            *cc.calculate_pre_collar_depths(wbd.depth_to_aquifer_base),
            wbd.calculate_pre_collar_casing_diameter()
        ]


    def export_to_json(self, filename):
        pass

In [4]:
depth_data = {
    "aquifer_layer": [
        "QA_UTQA",
        "UTQD",
        "UTAF",
        "UTD",
        "UMTA",
        "UMTD",
        "LMTA",
        "LTA",
        "BSE"
    ],
    "is_aquifer": [
        True,
        False,
        True,
        False,
        True,
        False,
        True,
        True,
        False
    ],
    "depth_to_base": [
        3,
        53,
        112,
        150,
        150,
        1000,
        1000,
        1221,
        1421
    ]
}

#depth = pd.DataFrame(depth_data)
initial_values = {
    "required_flow_rate": 4320,
    "hydraulic_conductivity": 5,
    "average_porosity": 0.25,
    "bore_lifetime_year": 30,
    "groundwater_depth": 25,
    "long_term_decline_rate": 1,
    "allowable_drawdown": 25,
    "safety_margin": 25
}

In [5]:
wbd = WellBoreDict()

wbd.initialise(depth_data=depth_data, **initial_values)

wbd.validate_initial_inputs()

calcpipe = CalcPipeline(wbd)
calcpipe.interval_pipeline()
print()
calcpipe.pump_pipeline()
print()
calcpipe.casing_pipeline()
print()
print(calcpipe.wbd.interval_data)

production_screen_length: 69.7420951749512
injection_screen_length: 139.4841903499024
production_screen_length_error: (62.767885657456084, 76.71630469244633)
injection_screen_length_error: (125.53577131491217, 153.43260938489266)
production_open_hole_diameter: 0.1905
injection_open_hole_diameter: 0.1905
best_option: 0.219075

pump_inlet_depth: 105
minimum_pump_housing_diameter: 0.28587049479391613

                     top  bottom  casing  drill_bit
casing_stages                                      
pre_collar           NaN     NaN     NaN        NaN
superficial_casing   NaN     NaN     NaN        NaN
pump_chamber_casing  NaN     NaN     NaN        NaN
intermediate_casing  NaN     NaN     NaN        NaN
screen_riser         NaN     NaN     NaN        NaN
production_screen    NaN     NaN     NaN        NaN

    prod_casing_diameters  casing_frictions  \
0                0.101600        564.263232   
1                0.114300        317.942443   
2                0.127000        190.322